In [33]:
import pyodbc
import pandas as pd
import numpy as np
from sodapy import Socrata

## Multas y Sanciones

In [73]:
def iniciarSesion():
    server = 'tcp:miadgpproyecto.database.windows.net' 
    database = 'DB_GP_PROYECTO' 
    username = 'admin_miad' 
    password = 'Abc123456789' 
    
    cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
    cursor = cnxn.cursor()
    
    return cnxn, cursor

In [74]:
def leerDatos():
    client = Socrata("www.datos.gov.co", None)
    results = client.get("it5q-hg94", limit=7000000)
    results_df = pd.DataFrame.from_records(results)
    
    return results_df

In [75]:
def borrarMultasYSanciones():
    cnxn.execute("""TRUNCATE TABLE Test_Secop2_MultasYSanciones""")
    cnxn.commit()

In [61]:
def cargueMultasYSanciones(cnxn, cursor):
    print('Proceso iniciado')
    
    cnxn.execute("""
    TRUNCATE TABLE Test_Secop2_MultasYSanciones""")
    cnxn.commit()
    print('Datos borrados')
    
    for index, row in results_df.iterrows():
        cursor.execute("""INSERT INTO Test_Secop2_MultasYSanciones ([ID Proceso], [Referencia Proceso], [ID Contrato],
           [Codigo Entidad Creadora], [Nombre Entidad Creadora],
           [ Codigo Proveedor Objeto de la Multa],
           [Nombre Proveedor Objeto de la Multa], [Valor], [Valor Pagado],
           [Fecha Evento], [Aplico Garantias], [Numero de Acto], [Tipo de Sancion],
           [Descripcion Otro Tipo De Sancion], [Estado], [Tipo],
           [Numero de Version]) values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)""", str(row.id_proceso), str(row.referencia_proceso), str(row.id_contrato)
                       , str(row.codigo_entidad_creadora), str(row.nombre_entidad_creadora)
                       , str(row.as_codigo_proveedor_objeto)
                       , str(row.nombre_proveedor_objeto_de), str(row.valor), str(row.valor_pagado)
                       , str(row.fecha_evento), str(row.aplico_garantias), str(row.numero_de_acto), str(row.tipo_de_sancion)
                       , str(row.descripcion_otro_tipo_de), str(row.estado), str(row.tipo)
                       , str(row.numero_de_version))
    cnxn.commit()
    
    print('Datos cargados')

In [76]:
def cerrarCursor(cursor):
    try:
        cursor.close()
    except:
        pass

In [50]:
# 1. Se inicia la conexion con la base de datos
cnxn, cursor = iniciarSesion()

# 2. Se cargan los datos desde el API
results_df = leerDatos()

# 3. Se limipia la tabla para actualizar con los nuevos datos
borrarMultasYSanciones()

# 4. Se cargan los datos completos a la base SQL
cargueMultasYSanciones(cnxn, cursor)

# 5. Se cierra el cursor
cerrarCursor(cursor)

### Prueba de lectura

In [51]:
sql = """
SELECT *
FROM Test_Secop2_MultasYSanciones
"""

In [68]:
data = pd.read_sql(sql,cnxn)

In [69]:
data.head()

,ID Proceso,Referencia Proceso,ID Contrato,Codigo Entidad Creadora,Nombre Entidad Creadora,Codigo Proveedor Objeto de la Multa,Nombre Proveedor Objeto de la Multa,Valor,Valor Pagado,Fecha Evento,Aplico Garantias,Numero de Acto,Tipo de Sancion,Descripcion Otro Tipo De Sancion,Estado,Tipo,Numero de Version
0,CO1.BDOS.1462517,INS-CYT-326-2020,nan,700927064,INSTITUTO NACIONAL DE SALUD,7348865,EXOGENA LTDA,6737683.200000,0.000000,2021-03-02T00:00:00.000,False,127-2021,Tipo Sancion,nan,Estado,Tipo,3
1,CO1.BDOS.652069,CM-PDA-002-2019,nan,700566128,EMPRESAS PUBLICAS DE CUNDINAMARCA S.A. ESP,5015870,GESTION INTEGRAL CONSULTORES SAS,12216540.000000,0.000000,2019-11-01T00:00:00.000,True,RESOLUCION 256 DE 2019,Tipo Sancion,nan,Estado,Tipo,2
2,CO1.BDOS.444114,030-00-T-GAAMA-ESALO-2018,CO1.PCCNTR.502494,702024282,GRUPO AEREO DEL AMAZONAS,7031560,NEOGLOBAL SAS,6690240.000000,0.000000,2018-12-06T00:00:00.000,False,RESOLUCIÓN NÚMERO N°2,Tipo Sancion,nan,Estado,Tipo,3
3,CO1.BDOS.910853,PN DIRAF MIC 161,nan,700569072,POLICÍA NACIONAL - DIRECCIÓN ADMINISTRATIVA Y ...,4418067,FM Solutions SAS,1117005.000000,0.000000,2020-03-06T00:00:00.000,True,Resolución No 0001 del 06/03/2020,Tipo Sancion,nan,Estado,Tipo,3
4,CO1.BDOS.2274246,MC-TLM-CIC-491-2021,CO1.PCCNTR.2933569,702561986,SENA REGIONAL TOLIMA 1,9403791,NOVATEK DEL CARIBE SAS,nan,nan,nan,nan,nan,Tipo Sancion,nan,Estado,Tipo,3


In [70]:
data.shape

(486, 17)